<a href="https://colab.research.google.com/github/khoai0ngu/Ung-Dung-Phan-Tan/blob/main/Chord_WebService.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# chord_sim.py
# Minimal Chord DHT simulation (single process, no real network)

import hashlib
from typing import Optional, Dict, List

M = 6  # số bit cho không gian identifier (0..2^M - 1)
RING_SIZE = 2 ** M


def sha1_hash(key: str) -> int:
    h = hashlib.sha1(key.encode()).hexdigest()
    return int(h, 16) % RING_SIZE


def in_interval(val: int, left: int, right: int, inclusive_right=False) -> bool:
    """Kiểm tra val có nằm trong khoảng (left, right) trên vòng tròn không."""
    if left == right:
        return True if inclusive_right else False
    if left < right:
        return (left < val <= right) if inclusive_right else (left < val < right)
    # wrap-around
    return (val > left and val < RING_SIZE) or (val < right) or (inclusive_right and val == right)


class Node:
    def __init__(self, node_id: int, network: 'Network'):
        self.id = node_id
        self.network = network
        self.successor: 'Node' = self
        self.predecessor: Optional['Node'] = None
        self.finger: List[Optional['Node']] = [None] * M
        self.data: Dict[int, str] = {}  # key_id -> value

    def __repr__(self):
        return f"Node({self.id})"

    def find_successor(self, id_: int) -> 'Node':
        if self == self.successor:
            return self
        if in_interval(id_, self.id, self.successor.id, inclusive_right=True):
            return self.successor
        n0 = self.closest_preceding_node(id_)
        if n0 == self:
            return self
        return n0.find_successor(id_)

    def closest_preceding_node(self, id_: int) -> 'Node':
        for i in range(M - 1, -1, -1):
            f = self.finger[i]
            if f and in_interval(f.id, self.id, id_):
                return f
        return self

    def join(self, known: Optional['Node']):
        if known:
            self.init_finger_table(known)
            self.update_others()
        else:
            for i in range(M):
                self.finger[i] = self
            self.predecessor = None
            self.successor = self

    def init_finger_table(self, known: 'Node'):
        start = (self.id + 2 ** 0) % RING_SIZE
        self.finger[0] = known.find_successor(start)
        self.successor = self.finger[0]
        self.predecessor = self.successor.predecessor
        self.successor.predecessor = self
        for i in range(M - 1):
            start = (self.id + 2 ** (i + 1)) % RING_SIZE
            if in_interval(start, self.id, self.finger[i].id, inclusive_right=True):
                self.finger[i + 1] = self.finger[i]
            else:
                self.finger[i + 1] = known.find_successor(start)

    def update_others(self):
        for i in range(M):
            pred_id = (self.id - 2 ** i + RING_SIZE) % RING_SIZE
            p = self.network.find_predecessor(pred_id)
            if p and p != self:
                p.update_finger_table(self, i)

    def update_finger_table(self, s: 'Node', i: int):
        if self.finger[i] is None or in_interval(s.id, self.id, self.finger[i].id):
            self.finger[i] = s
            p = self.predecessor
            if p and p != self:
                p.update_finger_table(s, i)

    def stabilize(self):
        x = self.successor.predecessor
        if x and in_interval(x.id, self.id, self.successor.id):
            self.successor = x
        self.successor.notify(self)

    def notify(self, n: 'Node'):
        if self.predecessor is None or in_interval(n.id, self.predecessor.id, self.id):
            self.predecessor = n

    def fix_fingers(self):
        for i in range(M):
            start = (self.id + 2 ** i) % RING_SIZE
            self.finger[i] = self.find_successor(start)

    def put(self, key: str, value: str):
        k = sha1_hash(key)
        node = self.find_successor(k)
        node.data[k] = value
        return node

    def get(self, key: str) -> Optional[str]:
        k = sha1_hash(key)
        node = self.find_successor(k)
        return node.data.get(k, None)


class Network:
    def __init__(self):
        self.nodes: Dict[int, Node] = {}

    def add_node(self, node: Node, known_node: Optional[Node] = None):
        self.nodes[node.id] = node
        node.join(known_node)
        # chạy stabilize/fix_fingers vài vòng để hội tụ
        for _ in range(3):
            for n in list(self.nodes.values()):
                n.stabilize()
            for n in list(self.nodes.values()):
                n.fix_fingers()

    def remove_node(self, node_id: int):
        if node_id in self.nodes:
            node = self.nodes.pop(node_id)
            # chuyển dữ liệu sang successor
            succ = node.successor
            succ.data.update(node.data)
            # cập nhật liên kết
            for n in self.nodes.values():
                if n.predecessor == node:
                    n.predecessor = node.predecessor
                if n.successor == node:
                    n.successor = node.successor
                for i in range(M):
                    if n.finger[i] == node:
                        n.finger[i] = node.successor

    def find_predecessor(self, id_: int) -> Optional[Node]:
        if not self.nodes:
            return None
        sorted_ids = sorted(self.nodes.keys())
        pred = None
        for nid in sorted_ids:
            if nid <= id_:
                pred = self.nodes[nid]
        if pred:
            return pred
        return self.nodes[sorted_ids[-1]]

    def dump(self):
        for n in sorted(self.nodes.values(), key=lambda x: x.id):
            fingers = [f.id if f else None for f in n.finger]
            print(
                f"Node {n.id}: succ={n.successor.id}, "
                f"pred={(n.predecessor.id if n.predecessor else None)}, "
                f"fingers={fingers}, data_keys={[k for k in n.data]}"
            )


# Demo
if __name__ == '__main__':
    net = Network()
    ids = [10, 22, 35]
    nodes = []
    for i in ids:
        n = Node(i, net)
        net.add_node(n, nodes[0] if nodes else None)
        nodes.append(n)

    net.dump()
    print('Put key serviceA -> 10.0.0.1:5000 via node 10')
    nodes[0].put('serviceA', '10.0.0.1:5000')
    print('Get serviceA via node 22 ->', nodes[1].get('serviceA'))
    net.dump()


Node 10: succ=10, pred=35, fingers=[10, 10, 10, 10, 10, 10], data_keys=[]
Node 22: succ=35, pred=10, fingers=[35, 35, 35, 35, 10, 10], data_keys=[]
Node 35: succ=10, pred=22, fingers=[10, 10, 10, 10, 10, 10], data_keys=[]
Put key serviceA -> 10.0.0.1:5000 via node 10
Get serviceA via node 22 -> 10.0.0.1:5000
Node 10: succ=10, pred=35, fingers=[10, 10, 10, 10, 10, 10], data_keys=[50]
Node 22: succ=35, pred=10, fingers=[35, 35, 35, 35, 10, 10], data_keys=[]
Node 35: succ=10, pred=22, fingers=[10, 10, 10, 10, 10, 10], data_keys=[]
